# Report Generation Tutorial

PyGSTi is able to construct polished report documents, which provide high-level summaries as well as detailed analyses  of GST results.  Reports are intended to be quick and easy way of analyzing a GST estimate, and pyGSTi's report generation functions are specifically designed to interact with its high-level driver functions (see the high-level algorithms tutorial).  Currently there is only a single report generation function, `pygsti.report.create_general_report`, which takes one or more `Results` objects as input and produces an HTML file as output.  The HTML format allows the reports to include **interactive plots** and **switches**, making it easy to compare different types of analysis or data sets.  

PyGSTi's "general" report creates a stand-alone HTML document which cannot run Python.  Thus, all the results displayed in the report must be pre-computed (in Python).  If you find yourself wanting to fiddle with things and feel that the general report is too static, please consider using a `Workspace` object (see following tutorials) within a Jupyter notebook, where you can intermix report tables/plots and Python.  Internally, `create_general_report` is just a canned routine which uses a `WorkSpace` object to generate various tables and plots and then inserts them into a HTML template.  

**Note to veteran users:** PyGSTi has recently transitioned to producing HTML (rather than LaTeX/PDF) reports.  The way to generate such report is largely unchanged, with one important exception.  Previously, the `Results` object had various report-generation methods included within it.  We've found this is too restrictive, as we'd sometimes like to generate a report which utilizes the results from multiple runs of GST (to compare the, for instance).  Thus, the `Results` class is now just a container for a `DataSet` and its related `GateSet`s, `GatestringStructure`s, etc.  All of the report-generation capability is now housed in within separate report functions, which we now demonstrate.


### Get some `Results`
We start by performing GST using `do_long_sequence_gst`, as usual, to create a `Results` object (we could also have just loaded one from file).

In [1]:
import pygsti
from pygsti.construction import std1Q_XYI

gs_target = std1Q_XYI.gs_target
fiducials = std1Q_XYI.fiducials
germs = std1Q_XYI.germs
maxLengths = [1,2,4,8,16]
ds = pygsti.io.load_dataset("tutorial_files/Example_Dataset.txt", cache=True)

#Run GST
gs_target.set_all_parameterizations("TP") #TP-constrained
results = pygsti.do_long_sequence_gst(ds, gs_target, fiducials, fiducials, germs,
                                      maxLengths, verbosity=3)

Loading from cache file: tutorial_files/Example_Dataset.txt.cache
--- Gate Sequence Creation ---
 1201 sequences created
 Dataset has 3121 entries: 1201 utilized, 0 requested sequences were missing
--- LGST ---
  Singular values of I_tilde (truncating to first 4 of 6) = 
  4.24503058336
  1.17971057338
  0.956497891831
  0.942353526676
  0.0470890214285
  0.0153149329552
  
  Singular values of target I_tilde (truncating to first 4 of 6) = 
  4.24264068712
  1.41421356237
  1.41421356237
  1.41421356237
  3.03015102482e-16
  7.09307430818e-17
  
    Resulting gate set:
    
    rho0 =    0.7071  -0.0302   0.0396   0.7480
    
    
    E0 =    0.7264  -0.0062   0.0037   0.6492
    
    
    Gi = 
       1.0000        0        0        0
       0.0094   0.9238   0.0542  -0.0155
       0.0285  -0.0149   0.9021   0.0200
      -0.0142   0.0280   0.0009   0.9057
    
    
    Gx = 
       1.0000        0        0        0
       0.0064   0.9053   0.0281  -0.0044
      -0.0006   0.0215  -0.04

### Make the reports
Now that we have `results`, we use the methods within `pygsti.report.factory` to generate reports.  These reports are HTML pages that you can open in your browser or email to your friends.  Setting `auto_open=True` makes the finished report open in your web browser automatically.  

In [2]:
pygsti.report.create_general_report(results, "tutorial_files/exampleGenReport.html", 
                                    title="GST Example Report", confidenceLevel=95, verbosity=2, auto_open=True)

*** Creating workspace ***
*** Generating switchboard tables ***
    
--- Hessian Projector Optimization for gate CIs (L-BFGS-B) ---
   10s           0.0281509441
   15s           0.0278846005
   20s           0.0278591627
   25s           0.0278428572
   30s           0.0278372063
   35s           0.0278361588
   40s           0.0278347895
   45s           0.0278341597
  The resulting min sqrt(sum(gateCIs**2)): 0.0278342
  targetSpamBriefTable                          took 0.000859 seconds
  targetGatesBoxTable                           took 0.014751 seconds
  datasetOverviewTable                          took 0.059289 seconds
  bestGatesetSpamParametersTable                took 0.007657 seconds
  bestGatesetSpamBriefTable                     took 0.001428 seconds
  bestGatesetSpamVsTargetTable                  took 0.014718 seconds
  bestGatesetGaugeOptParamsTable                took 0.000245 seconds
  bestGatesetGatesBoxTable                      took 0.026808 seconds
  bestGatesetC

There are several features of the report worth noting:
1. Moving your mouse over to the far left edge reveals a navigation menu.  Use this menu to change to a different "tab" (section) of the report.
2. These HTML reports are created quickly but require some time to load within a browser.  This is because the typsetting and figure rendering is done within your browser, rather than up front (by `pdflatex` in earlier versions of pyGSTi).
3. The plots are interactive - a primary advantage of the HTML format.
4. Plots that show up as a big PyGSTi **pig icon** are those whose rendering has been deferred because they can take a little while to display. **The pig icon can be clicked to trigger rendering of these plots**.

Next, let's analyze the same data two different ways: with and without the TP-constraint (i.e. whether the gates *must* be trace-preserving) and furthermore gauge optmimize each case using several different SPAM-weights.  In each case we'll call `do_long_sequence_gst` with `gaugeOptParams=False`, so that no gauge optimization is done, and then perform several gauge optimizations separately and add these to the `Results` object via its `add_gaugeoptimized` function.

In [3]:
#Case1: TP-constrained GST
tpTarget = gs_target.copy()
tpTarget.set_all_parameterizations("TP")
results_tp = pygsti.do_long_sequence_gst(ds, tpTarget, fiducials, fiducials, germs,
                                      maxLengths, gaugeOptParams=False, verbosity=1)

#Gauge optimize
est = results_tp.estimates['default']
gsFinal = est.gatesets['final iteration estimate']
gsTarget = est.gatesets['target']
for spamWt in [1e-4,1e-2,1.0]:
    gs = pygsti.gaugeopt_to_target(gsFinal,gsTarget,{'gates':1, 'spam':spamWt})
    est.add_gaugeoptimized({'itemWeights': {'gates':1, 'spam':spamWt}}, gs, "Spam %g" % spamWt)

--- Gate Sequence Creation ---
 1201 sequences created
 Dataset has 3121 entries: 1201 utilized, 0 requested sequences were missing
--- LGST ---
--- Iterative MLGST: [##################################################] 100.0%  1201 gate strings ---
Iterative MLGST Total Time: 1.1s


In [4]:
#Case2: "Full" GST
fullTarget = gs_target.copy()
fullTarget.set_all_parameterizations("full")
results_full = pygsti.do_long_sequence_gst(ds, fullTarget, fiducials, fiducials, germs,
                                      maxLengths, gaugeOptParams=False, verbosity=1)

#Gauge optimize
est = results_full.estimates['default']
gsFinal = est.gatesets['final iteration estimate']
gsTarget = est.gatesets['target']
for spamWt in [1e-4,1e-2,1.0]:
    gs = pygsti.gaugeopt_to_target(gsFinal,gsTarget,{'gates':1, 'spam':spamWt})
    est.add_gaugeoptimized({'itemWeights': {'gates':1, 'spam':spamWt}}, gs, "Spam %g" % spamWt)

--- Gate Sequence Creation ---
 1201 sequences created
 Dataset has 3121 entries: 1201 utilized, 0 requested sequences were missing
--- LGST ---
--- Iterative MLGST: [##################################################] 100.0%  1201 gate strings ---
Iterative MLGST Total Time: 1.7s


We'll now call the *same* `create_general_report` function but this time instead of passing a single `Results` object as the first argument we'll pass a *dictionary* of them.  This will result in a report that includes switches to select which case ("TP" or "Full") as well as which gauge optimization to display output quantities for.

In [5]:
ws = pygsti.report.create_general_report({'TP': results_tp, "Full": results_full},
                                         "tutorial_files/exampleMultiGenReport.html",
                                         title="Example Multi-Dataset Report", 
                                         verbosity=3, auto_open=True)

*** Creating workspace ***
*** Generating switchboard tables ***
  targetSpamBriefTable                          took 0.001612 seconds
  targetGatesBoxTable                           took 0.029592 seconds
  datasetOverviewTable                          took 0.068667 seconds
  bestGatesetSpamParametersTable                took 0.002909 seconds
  bestGatesetSpamBriefTable                     took 0.007615 seconds
  bestGatesetSpamVsTargetTable                  took 0.014896 seconds
  bestGatesetGaugeOptParamsTable                took 0.00044 seconds
  bestGatesetGatesBoxTable                      took 0.088205 seconds
  bestGatesetChoiEvalTable                      took 0.039067 seconds
  bestGatesetDecompTable                        took 0.042378 seconds
  bestGatesetEvalTable                          took 0.095441 seconds
  bestGatesetRelEvalTable                       took 0.00445 seconds
Predicted RB decay parameter / error rate may be unreliable:
Gateset is not (approximately) trace

In the above call we capture the return value in the variable `ws` - a `Workspace` object.  PyGSTi's `Workspace` objects function as both a factory for figures and tables as well as a smart cache for computed values.  Within `create_general_report` a `Workspace` object is created and used to create all the figures in the report.  As an intended side effect, each of these figures is cached, along with some of the intermediate results used to create it.  As we'll see below, a `Workspace` can also be specified as input to `create_general_report`, allowing it to utilize previously cached quantities.

**Note to veteran users:** Other report formats such as **`beamer`-class PDF presentation and Powerpoint presentation have been dropped from pyGSTi**.  These presentation formats were rarely used and moreover we feel that the HTML format is able to provide all of the functionality that was present in these discontinued formats.

### Sharing with others
The report factory functions may be passed a boolean `connected` argument to specify whether online (`True`) or local (`False`) resources should be used by the generated report HTML files.  If `connected=False` (the default, and so what we used in this notebook), then the HTML files rely on the presence of an `offline` folder being in their directory to operate correctly.  The offline folder is automatically generated for you when you call any of the report factory functions, so when viewing reports on the same machine that generates them you typically don't need to worry about it.  

However, *sharing offline reports* (created with `connected=False`) requires that the recipient has pyGSTi's `offline` folder in the same diretory as any report HTML files.  To facilitate sharing, you can get a zipped copy of the offline folder by calling `create_offline_zip` like this:

In [6]:
pygsti.report.create_offline_zip("tutorial_files/")
  #creates .../tutorial_files/offline.zip

If you use `connected=True`, then there's no need for the offline folder -- but you'll need to have an internet connection to load the HTML file.

### Report HTML files too large?
Sometimes when you're dealing with a large data set or many different estimates and/or gauge optimizations, the report file size becomes unwieldy (100+ MB HTML files!).  This is due to the reports containing the entire data set and various computed values for *each* experiment that was performed - for each estimate and each data set that is being reported.  To help mitigate this, `create_general_report` accepts the `brief` argument, which, when set to `True` (its default is `False`) omits the most data-hungry figures and tables of the report.  This results in a report in which the omitted figures and tables are replaced with "**BLANK**", and an HTML file size which is significantly smaller. 

In [7]:
pygsti.report.create_general_report({'TP': results_tp, "Full": results_full},
                                    "tutorial_files/exampleMultiGenBriefReport.html",
                                    title="Example Multi-Dataset Brief Report", ws=ws,
                                    verbosity=3, auto_open=True, brief=True)

*** Creating workspace ***
*** Generating switchboard tables ***
  targetSpamBriefTable                          took 0.000303 seconds
  targetGatesBoxTable                           took 0.00023 seconds
  datasetOverviewTable                          took 0.000226 seconds
  bestGatesetSpamParametersTable                took 0.000443 seconds
  bestGatesetSpamBriefTable                     took 0.000702 seconds
  bestGatesetSpamVsTargetTable                  took 0.000521 seconds
  bestGatesetGaugeOptParamsTable                took 0.000341 seconds
  bestGatesetGatesBoxTable                      took 0.001084 seconds
  bestGatesetChoiEvalTable                      took 0.000569 seconds
  bestGatesetDecompTable                        took 0.000492 seconds
  bestGatesetEvalTable                          took 0.001064 seconds
  bestGatesetRelEvalTable                       took 0.000549 seconds
  bestGatesetVsTargetTable                      took 0.000531 seconds
  bestGatesVsTargetTable  

The above call also demonstrates the use of the `ws` argument, by which we can give `create_general_report` an *input* `Workspace` object to use instead of internally creating a new one.  This allows the report to generate more quickly because cached figures and intermediate results are used.

## Advanced Reports: `create_report_notebook`
In addition to the standard HTML-page reports demonstrated above, pyGSTi is able to generate a Jupyter notebook containing the Python commands to create the figures and tables within a general report.  This is facilitated
by `Workspace` objects, which are factories for figures and tables (see previous tutorials).  By calling `create_report_notebook`, all of the relevant `Workspace` initialization and calls are dumped to a new notebook file, which can be run (either fully or partially) by the user at their convenience.  Creating such "report notebooks" has the advantage that the user may insert Python code amidst the figure and table generation calls to inspect or modify what is display in a highly customizable fashion.  The chief disadvantages of report notebooks is that they require the user to 1) have a Jupyter server up and running and 2) to run the notebook before any figures are displayed.

The line below demonstrates how to create a report notebook using `create_report_notebook`.  Note that the argument list is very similar to `create_general_report`.

In [8]:
pygsti.report.create_report_notebook(results, "tutorial_files/exampleGenReport.ipynb", 
                                     title="GST Example Report Notebook", confidenceLevel=None,
                                     auto_open=True, connected=False, verbosity=3)